In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
plt.rcParams['font.family'] = 'AppleGothic'

In [2]:
year_code = '2024'
month_code = '11'
recent_month_code = str(int(month_code)-1)
month_day = 30
recent_month = pd.read_excel('/Users/user/Documents/BOA 데이터/boa24'+recent_month_code+'.xlsx')
recent_month = recent_month[['회원번호','구분3','paid user 여부','매출금액(당월)','이용금액(당월)','파트너명','채널']]
recent_month = recent_month[recent_month['paid user 여부']=='paid']
recent_month1 = recent_month.groupby(['회원번호','구분3'])[['매출금액(당월)','이용금액(당월)']].sum().reset_index()
recent_month1 = pd.merge(recent_month1, recent_month[['회원번호','구분3','paid user 여부','파트너명','채널']], how = 'left',on = ['회원번호','구분3'])
recent_month = recent_month1
recent_month = recent_month.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
recent_month

,회원번호,구분3,매출금액(당월),이용금액(당월),paid user 여부,파트너명,채널
0,734,NAVER WORKS,104983,140000,paid,-,영업 매출
1,916,NAVER WORKS,53999,63000,paid,-,영업 매출
2,1084,NAVER WORKS,96189,112220,paid,아이지니 주식회사,파트너 매출
3,1098,NAVER WORKS,6000,7000,paid,-,영업 매출
4,1279,NAVER WORKS,6000,7000,paid,-,영업 매출
...,...,...,...,...,...,...,...
29621,3418303,WORKPLACE,2997,4260,paid,-,온라인 매출
29622,3418683,NAVER WORKS,7000,8500,paid,-,온라인 매출
29623,3419003,NAVER WORKS,26004,35090,paid,-,온라인 매출
29624,3419003,WORKPLACE,18595,28890,paid,-,온라인 매출


In [4]:
ncd_raw = pd.read_csv('/Users/user/Documents/진행중 업무/ncd_metering2.csv')

/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_84815/2891095201.py:1: DtypeWarning: Columns (31,35,36,41,43,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  ncd_raw = pd.read_csv('/Users/user/Documents/진행중 업무/ncd_metering2.csv')


In [5]:
ncd = ncd_raw.copy()
ncd = ncd[ncd['회원그룹명'] != '내부회원']
ncd = ncd[ncd['사용량'] != 0]
ncd = ncd[ncd['요금'] != 0]
#ncd = ncd[['회원번호','회원그룹코드','파트너사회원번호','plan','미터링일시','요금','사용량','약정여부','regist_ymd','delete_ymd']]
ncd['month_info'] = ncd['미터링일시'].astype(str).str[0:6]

In [6]:
ncd = ncd.sort_values(by=['회원번호', 'plan', '미터링일시'])
ncd['월매출(당월)'] = ncd.groupby(['회원번호', 'plan','month_info'])['일매출(당일)'].transform('sum')
ncd['월요금(당월)'] = ncd.groupby(['회원번호', 'plan','month_info'])['일요금(당일)'].transform('sum')
ncd

,회원번호,회원그룹코드,회원그룹명,파트너사회원번호,청구유형코드,청구유형코드명,청구유형상세코드,청구유형상세코드명,인스턴스유형코드,인스턴스유형코드명,...,도메인구독상품_구분3,도메인WORKPLACE유료구독상세상품_구분3,도메인WORKPLACE구독상세상품_구분3,이전상품_Trial포함,Paid_이전상품,일요금(당일),일매출(당일),month_info,월매출(당월),월요금(당월)
1275,734,GEN,Ncloud온라인회원,NaN,WORKS,NAVER WORKS (Deprecated),WORKS_LITE_FCHG,NAVER WORKS LITE 유료,WORKS,NAVER WORKS (Deprecated),...,웍스코어,NaN,NaN,NaN,NaN,4266.666667,3200.000000,202304,44800.000000,59733.333333
1276,734,GEN,Ncloud온라인회원,NaN,WORKS,NAVER WORKS (Deprecated),WORKS_LITE_FCHG,NAVER WORKS LITE 유료,WORKS,NAVER WORKS (Deprecated),...,웍스코어,NaN,NaN,NaN,NaN,4266.666667,3200.000000,202304,44800.000000,59733.333333
1277,734,GEN,Ncloud온라인회원,NaN,WORKS,NAVER WORKS (Deprecated),WORKS_LITE_FCHG,NAVER WORKS LITE 유료,WORKS,NAVER WORKS (Deprecated),...,웍스코어,NaN,NaN,NaN,NaN,4266.666667,3200.000000,202304,44800.000000,59733.333333
1278,734,GEN,Ncloud온라인회원,NaN,WORKS,NAVER WORKS (Deprecated),WORKS_LITE_FCHG,NAVER WORKS LITE 유료,WORKS,NAVER WORKS (Deprecated),...,웍스코어,NaN,NaN,NaN,NaN,4266.666667,3200.000000,202304,44800.000000,59733.333333
1279,734,GEN,Ncloud온라인회원,NaN,WORKS,NAVER WORKS (Deprecated),WORKS_LITE_FCHG,NAVER WORKS LITE 유료,WORKS,NAVER WORKS (Deprecated),...,웍스코어,NaN,NaN,NaN,NaN,4266.666667,3200.000000,202304,44800.000000,59733.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26149482,3448061,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,웍스코어,NaN,NaN,NaN,NaN,1370.967742,1209.677419,202412,3629.032258,4112.903226
26149596,3448178,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,웍스코어,NaN,NaN,NaN,NaN,548.387097,483.870968,202412,967.741935,1096.774194
26149597,3448178,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,웍스코어,NaN,NaN,NaN,NaN,548.387097,483.870968,202412,967.741935,1096.774194
26149697,3448298,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_SPLUS_USER_CNT,웍스 드라이브 Standard Plus,WORK2,NAVER WORKS,...,"웍스코어, 드라이브",NaN,NaN,동시구독,NaN,451.612903,387.096774,202412,774.193548,903.225806


In [8]:
ncd_unique = ncd.drop_duplicates(subset=['회원번호', 'plan','month_info'], keep='first').reset_index(drop=True)
ncd_unique['회원번호'] = ncd_unique['회원번호'].astype(int)
ncd_unique['구분3'] = np.where(ncd_unique['plan'].str[0:3]=='COR','NAVER WORKS',
                             np.where(ncd_unique['plan'].str[0:3].isin(['DRV','BOX']),'WORKS DRIVE',
                                     np.where(ncd_unique['plan'].str[0:3].isin(['APP','ATT','FIN','PAY']),'WORKPLACE','클로바노트')))
ncd_unique1 = ncd_unique.groupby(['회원번호','회원그룹코드','구분3','month_info'])[['월매출(당월)','월요금(당월)']].sum().reset_index()
ncd_unique1 = pd.merge(ncd_unique1, ncd_unique[['회원번호','파트너사회원번호','plan','미터링일시','요금','사용량','약정여부','regist_ymd','delete_ymd','구분3']], how = 'left',on = ['회원번호','구분3'])
ncd_unique = ncd_unique1
ncd_unique1 = ncd_unique1.drop_duplicates(subset=['회원번호', 'plan'], keep='first').reset_index(drop=True)
ncd_unique = ncd_unique[ncd_unique['구분3']!='클로바노트']
ncd_unique1 = ncd_unique1[ncd_unique1['구분3']!='클로바노트']
ncd_unique = ncd_unique.reset_index(drop=True)
ncd_unique1 = ncd_unique1.reset_index(drop=True)

In [9]:
def reverse_format_date(x):
    try:
        year, month, day = x.split('.')
        return int(round(int(f"{year}{int(month):02}{int(day):02}")))
    except (ValueError, AttributeError):
        return None

ncd_unique['등록일시'] = ncd_unique['regist_ymd'].apply(reverse_format_date)
ncd_unique['탈퇴일시'] = ncd_unique['delete_ymd'].apply(reverse_format_date)
ncd_unique1['등록일시'] = ncd_unique1['regist_ymd'].apply(reverse_format_date)
ncd_unique1['탈퇴일시'] = ncd_unique1['delete_ymd'].apply(reverse_format_date)

In [51]:
ncd_new = ncd_unique[ncd_unique['등록일시'].astype(str).str[0:6] == str(int((int(year_code)*10000 + int(month_code)*100)/100))]
ncd_new = ncd_new[ncd_new['요금']>0]
ncd_last_churn = ncd_unique[ncd_unique['탈퇴일시'].astype(str).str[0:6] == str(int((int(year_code)*10000 + (int(month_code)-1)*100)/100))]
ncd_last_churn = ncd_last_churn[ncd_last_churn['요금']>0]
ncd_churn = ncd_unique[ncd_unique['탈퇴일시'].astype(str).str[0:6] == str(int((int(year_code)*10000 + int(month_code)*100)/100))]
ncd_churn = ncd_churn[ncd_churn['요금']>0]

In [11]:
current_month = recent_month
current_month = current_month[~current_month.set_index(['회원번호', '구분3']).index.isin(
    ncd_last_churn.set_index(['회원번호', '구분3']).index
)]

In [12]:
ncd_churn1 = ncd_churn.groupby(['회원번호','구분3'])[['월매출(당월)','월요금(당월)']].sum().reset_index()
ncd_churn1 = pd.merge(ncd_churn1, ncd_churn[['회원번호','구분3','등록일시','탈퇴일시','파트너사회원번호','사용량']], how = 'left',on = ['회원번호','구분3'])
ncd_churn = ncd_churn1
ncd_churn = ncd_churn.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
current_month = pd.merge(current_month, ncd_churn, on = ['회원번호','구분3'], how = 'left')
current_month[current_month['탈퇴일시']>0]

,회원번호,구분3,매출금액(당월),이용금액(당월),paid user 여부,파트너명,채널,월매출(당월),월요금(당월),등록일시,탈퇴일시,파트너사회원번호,사용량
8,1352,NAVER WORKS,54000,63000,paid,주식회사 페이지원,파트너 매출,2.962313e+07,3.437513e+07,20230101.0,20241128.0,2673247.0,9.0
184,2535624,NAVER WORKS,4900,6030,paid,-,온라인 매출,1.214516e+04,1.376452e+04,20241010.0,20241104.0,NaN,1.0
242,2541684,WORKPLACE,8051,11970,paid,-,영업 매출,5.144111e+07,5.925720e+07,20230101.0,20241111.0,NaN,3.0
244,2541689,WORKPLACE,18900,28000,paid,-,온라인 매출,3.870790e+07,4.488340e+07,20230101.0,20241104.0,NaN,7.0
363,2550462,NAVER WORKS,32400,42000,paid,-,온라인 매출,2.960193e+07,3.837288e+07,20230101.0,20241103.0,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29008,3401037,WORKPLACE,26639,39480,paid,-,온라인 매출,1.328632e+05,1.968344e+05,20241023.0,20241114.0,NaN,34.0
29024,3401648,NAVER WORKS,16200,19740,paid,-,온라인 매출,4.233871e+04,4.798387e+04,20241008.0,20241106.0,NaN,3.0
29027,3401788,NAVER WORKS,4000,4930,paid,-,온라인 매출,8.224688e+04,9.680645e+04,20241014.0,20241107.0,NaN,1.0
29158,3406756,NAVER WORKS,9823,11740,paid,-,온라인 매출,3.470806e+04,4.370645e+04,20241023.0,20241118.0,NaN,3.0


In [13]:
current_month['이용금액(당월)'] = current_month['이용금액(당월)'].astype(float)
current_month['매출금액(당월)'] = current_month['매출금액(당월)'].astype(float)
current_month.loc[current_month['탈퇴일시']>0,'이용금액(당월)'] = (current_month.loc[current_month['탈퇴일시']>0,'이용금액(당월)'])*((current_month.loc[current_month['탈퇴일시']>0,'탈퇴일시']-(int(year_code)*10000+int(month_code)*100))/month_day)
current_month.loc[current_month['탈퇴일시']>0,'매출금액(당월)'] = (current_month.loc[current_month['탈퇴일시']>0,'매출금액(당월)'])*((current_month.loc[current_month['탈퇴일시']>0,'탈퇴일시']-(int(year_code)*10000+int(month_code)*100))/month_day)

In [14]:
current_month[current_month['탈퇴일시']>0]

,회원번호,구분3,매출금액(당월),이용금액(당월),paid user 여부,파트너명,채널,월매출(당월),월요금(당월),등록일시,탈퇴일시,파트너사회원번호,사용량
8,1352,NAVER WORKS,50400.000000,58800.000000,paid,주식회사 페이지원,파트너 매출,2.962313e+07,3.437513e+07,20230101.0,20241128.0,2673247.0,9.0
184,2535624,NAVER WORKS,653.333333,804.000000,paid,-,온라인 매출,1.214516e+04,1.376452e+04,20241010.0,20241104.0,NaN,1.0
242,2541684,WORKPLACE,2952.033333,4389.000000,paid,-,영업 매출,5.144111e+07,5.925720e+07,20230101.0,20241111.0,NaN,3.0
244,2541689,WORKPLACE,2520.000000,3733.333333,paid,-,온라인 매출,3.870790e+07,4.488340e+07,20230101.0,20241104.0,NaN,7.0
363,2550462,NAVER WORKS,3240.000000,4200.000000,paid,-,온라인 매출,2.960193e+07,3.837288e+07,20230101.0,20241103.0,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29008,3401037,WORKPLACE,12431.533333,18424.000000,paid,-,온라인 매출,1.328632e+05,1.968344e+05,20241023.0,20241114.0,NaN,34.0
29024,3401648,NAVER WORKS,3240.000000,3948.000000,paid,-,온라인 매출,4.233871e+04,4.798387e+04,20241008.0,20241106.0,NaN,3.0
29027,3401788,NAVER WORKS,933.333333,1150.333333,paid,-,온라인 매출,8.224688e+04,9.680645e+04,20241014.0,20241107.0,NaN,1.0
29158,3406756,NAVER WORKS,5893.800000,7044.000000,paid,-,온라인 매출,3.470806e+04,4.370645e+04,20241023.0,20241118.0,NaN,3.0


In [15]:
current_month['매출금액(당월)'].sum()

2010093056.8999999

In [16]:
ncd_new[(ncd_new['month_info']=='202411')&(ncd_new['미터링일시'].astype(str).str[0:6]=='202411')]['월매출(당월)'].sum()

18207705.042598486

In [17]:
promotion = pd.read_excel('/Users/user/Documents/진행중 업무/웍스오픈프로모션_24.06-09대상자_241010_v3.xlsx', sheet_name=None)
june = list(promotion.keys())[1]
june_promotion = promotion[june]
june_promotion = june_promotion.iloc[1:8,0:6]
june_promotion['회원번호'] = june_promotion['회원번호'].astype(int)
july = list(promotion.keys())[2]
july_promotion = promotion[july]
july_promotion = july_promotion.iloc[1:,0:6]
#july_promotion = july_promotion.drop(411, axis=0)
july_promotion['회원번호'] = july_promotion['회원번호'].astype(int)
august = list(promotion.keys())[3]
august_promotion = promotion[august]
august_promotion = august_promotion.iloc[1:,0:6]
august_promotion['회원번호'] = august_promotion['회원번호'].astype(int)
september = list(promotion.keys())[4]
september_promotion = promotion[september]
september_promotion = september_promotion.iloc[1:,0:6]
september_promotion['회원번호'] = september_promotion['회원번호'].astype(int)

# june_promotion 제외
#temp_merge = merge.merge(june_promotion[['회원번호', 'base_item_id 상품명']], 
#                         left_on=['회원번호', 'plan'], 
#                         right_on=['회원번호', 'base_item_id 상품명'], 
#                         how='left', indicator='_merge1')
#merge = temp_merge[temp_merge['_merge1'] == 'left_only'].drop(columns=['_merge1', 'base_item_id 상품명'])

# july_promotion 제외
#temp_merge = merge.merge(july_promotion[['회원번호', 'base_item_id 상품명']], 
#                         left_on=['회원번호', 'plan'], 
#                         right_on=['회원번호', 'base_item_id 상품명'], 
#                          how='left', indicator='_merge2')
#merge = temp_merge[temp_merge['_merge2'] == 'left_only'].drop(columns=['_merge2', 'base_item_id 상품명'])

# august_promotion 제외
#temp_merge = merge.merge(august_promotion[['회원번호', 'base_item_id 상품명']], 
#                         left_on=['회원번호', 'plan'],
#                         right_on=['회원번호', 'base_item_id 상품명'], 
#                         how='left', indicator='_merge3')
#merge = temp_merge[temp_merge['_merge3'] == 'left_only'].drop(columns=['_merge3', 'base_item_id 상품명'])

# september_promotion 제외
#temp_merge = merge.merge(september_promotion[['회원번호', 'base_item_id 상품명']], 
#                         left_on=['회원번호', 'plan'], 
#                         right_on=['회원번호', 'base_item_id 상품명'],
#                         how='left', indicator='_merge4')
#merge = temp_merge[temp_merge['_merge4'] == 'left_only'].drop(columns=['_merge4', 'base_item_id 상품명'])

In [18]:
september_promotion

,회원번호,domain_id,base_item_id 상품명,list price,usercount,이용금액
1,2678955,226803.0,COR_LT,3000.0,26.7778,80333.4
2,2965498,300056727.0,COR_LT,3000.0,4.0000,12000.0
3,3180094,300109200.0,COR_STD,7000.0,3.2222,22555.4
4,3185229,300109826.0,COR_LT,3000.0,37.6667,113000.1
5,3248534,300125003.0,COR_STD,7000.0,2.0000,14000.0
...,...,...,...,...,...,...
669,3410268,300170087.0,COR_STD,7000.0,6.0000,42000.0
670,3344956,300151043.0,COR_STD,7000.0,14.0000,98000.0
671,3376413,300158530.0,APP_STD,3000.0,20.8889,62666.7
672,3376413,300158530.0,ATT_STD,2000.0,20.8889,41777.8


In [19]:
pro = ncd_unique[ncd_unique.set_index(['회원번호', 'plan']).index.isin(
    july_promotion.set_index(['회원번호', 'base_item_id 상품명']).index
)]

In [20]:
pro = pro[(pro['month_info']==str(int(year_code)*100+int(month_code)))&(pro['미터링일시'].astype(str).str[0:6]=='202411')]

In [21]:
pro['월매출(당월)'].sum()

13470066.722846441

In [39]:
pro

,회원번호,회원그룹코드,구분3,month_info,월매출(당월),월요금(당월),파트너사회원번호,plan,미터링일시,요금,사용량,약정여부,regist_ymd,delete_ymd,등록일시,탈퇴일시
446550,2575049,GEN,NAVER WORKS,202411,60000.0,80000.000000,NaN,COR_LT,20241101,4000.0,20.0,Y,2024.7.28,NaN,20240728,NaN
648391,2593792,GEN,NAVER WORKS,202411,7500.0,8500.000000,NaN,COR_STD,20241101,8500.0,1.0,Y,2024.7.2,NaN,20240702,NaN
901519,2617393,GEN,NAVER WORKS,202411,22500.0,25500.000000,NaN,COR_STD,20241101,8500.0,3.0,Y,2024.7.29,NaN,20240729,NaN
1062504,2639346,GEN,NAVER WORKS,202411,15000.0,17000.000000,NaN,COR_STD,20241101,8500.0,2.0,Y,2024.7.25,NaN,20240725,NaN
1222296,2650315,GEN,NAVER WORKS,202411,29000.0,32866.666667,NaN,COR_STD,20241101,8500.0,4.0,Y,2024.7.29,NaN,20240729,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16485089,3375718,GEN,NAVER WORKS,202411,7500.0,8500.000000,NaN,COR_STD,20241101,8500.0,1.0,Y,2024.7.20,NaN,20240720,NaN
16486429,3377143,GEN,NAVER WORKS,202411,24000.0,27200.000000,NaN,COR_STD,20241101,8500.0,3.0,Y,2024.7.30,NaN,20240730,NaN
16487780,3378721,GEN,NAVER WORKS,202411,15000.0,17000.000000,NaN,COR_STD,20241101,8500.0,2.0,Y,2024.7.23,NaN,20240723,NaN
16488057,3378958,GEN,NAVER WORKS,202411,11000.0,13000.000000,NaN,COR_STD_P,20241101,13000.0,1.0,Y,2024.7.23,NaN,20240723,NaN


In [41]:
ncd_new

,회원번호,회원그룹코드,구분3,month_info,월매출(당월),월요금(당월),파트너사회원번호,plan,미터링일시,요금,사용량,약정여부,regist_ymd,delete_ymd,등록일시,탈퇴일시
99350,2528554,GEN,NAVER WORKS,202411,7500.000000,8500.000000,NaN,COR_STD,20241101,8500.0,1.0,Y,2024.11.1,NaN,20241101,NaN
99351,2528554,GEN,NAVER WORKS,202411,7500.000000,8500.000000,NaN,COR_STD,20241201,8500.0,1.0,Y,2024.11.1,NaN,20241101,NaN
99352,2528554,GEN,NAVER WORKS,202412,6048.387097,6854.838710,NaN,COR_STD,20241101,8500.0,1.0,Y,2024.11.1,NaN,20241101,NaN
99353,2528554,GEN,NAVER WORKS,202412,6048.387097,6854.838710,NaN,COR_STD,20241201,8500.0,1.0,Y,2024.11.1,NaN,20241101,NaN
119093,2531615,PATR,NAVER WORKS,202411,10250.000000,11616.666667,2903722.0,COR_STD,20241114,8500.0,1.0,Y,2024.11.14,NaN,20241114,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16508651,3433349,GEN,WORKS DRIVE,202412,12903.225806,12903.225806,NaN,DRV_LT,20241201,4000.0,4.0,N,2024.11.22,NaN,20241122,NaN
16508665,3433674,GEN,WORKS DRIVE,202411,2400.000000,2400.000000,NaN,DRV_LT,20241122,4000.0,2.0,N,2024.11.22,NaN,20241122,NaN
16508666,3433674,GEN,WORKS DRIVE,202411,2400.000000,2400.000000,NaN,DRV_LT,20241201,4000.0,2.0,N,2024.11.22,NaN,20241122,NaN
16508667,3433674,GEN,WORKS DRIVE,202412,6451.612903,6451.612903,NaN,DRV_LT,20241122,4000.0,2.0,N,2024.11.22,NaN,20241122,NaN
